In [1]:
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from matplotlib import pyplot
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input, LSTM, Dense, RepeatVector
import tensorflow as tf
from keras.models import Sequential
import math
import enum
import os
from pathlib import Path
import argparse
from copy import deepcopy
from skimage.measure import compare_nrmse
import time
import hashlib
import json

### loader.py

In [2]:
def get_working_path():
    path = Path(os.getcwd()).parent
    return path


def get_child_dirs(path):
    directories = [f.path for f in os.scandir(path) if f.is_dir()]
    return directories


def get_child_files(path):
    files = [f.path for f in os.scandir(path) if f.is_file()]
    return files


def get_power_file(plant_number, date):
    dir_name = "UR00000%d" % plant_number
    file_name = date.strftime("%Y%m%d") + ".json"
    path = os.path.join(get_working_path(),
                              "data", "pow_24", dir_name, file_name)
    return path


def get_weather_file(spot_index, date):
    year = int(date.strftime("%Y"))
    file_name = "SURFACE_ASOS_%d_HR_%d_%d_%d.csv"\
                % (spot_index, year, year, year + 1)
    path = os.path.join(get_working_path(),
                        "data", "weather", file_name)
    return path


def insert_json_row(idx, df, df_insert):
    begin = df.iloc[:idx, ]
    end = df.iloc[idx:, ]
    result = pd.concat([begin, df_insert, end], ignore_index=True)
    return result


def check_json_time_series(json_data):
    for i in range(22):
        if int(json_data.loc[i]['result'].get('logHr')) != i:
            nan_row = pd.DataFrame.from_dict({"result": [{'hrPow': np.nan, 'logHr': "%02d" % i}]})
            json_data = insert_json_row(i, json_data, nan_row)
    new_row = pd.DataFrame.from_dict({"result": [{'hrPow': 0, 'logHr': "23"}]})
    json_data = insert_json_row(23, json_data, new_row)

    return json_data


def read_json(plant_number, date):
    path = get_power_file(plant_number, date)

    if os.path.isfile(path) is False or os.stat(path).st_size == 0:
        row = [pd.DataFrame.from_dict({"result": [{'hrPow': 0, 'logHr': '%02d' % i}]}) for i in range(24)]
        json_data = pd.concat(row, ignore_index=True)
    else:
        json_data = pd.read_json(path)
        json_data = check_json_time_series(json_data)

    json_data = pd.json_normalize(json_data['result'])

    return json_data


def read_csv(spot_index, date, duration=1):
    dates = [date + timedelta(days=i) for i in range(duration)]

    str_dates = []
    years = []
    result = None

    year = int(dates[0].strftime("%Y"))
    years.append(year)
    for i, date in enumerate(dates):
        str_date = date.strftime("%Y-%m-%d")
        new_year = int(date.strftime("%Y"))
        if year != new_year:
            years.append(new_year)
        year = new_year
        str_dates.append(str_date)

    for year in years:
        sub_str_dates = [str_date for str_date in str_dates if str(year) in str_date]
        path = get_weather_file(spot_index, datetime.strptime(sub_str_dates[0], "%Y-%m-%d"))
        csv_data = pd.read_csv(path, encoding='cp949')
        csv_data_days = [csv_data[csv_data['일시'].str.contains(sub_str_dates[i])] for i in range(len(sub_str_dates))]
        result = pd.concat(csv_data_days)

    return result

### util.py

In [3]:
def get_power_data(plant_number, date, duration=1):
    json_data = read_json(plant_number, date)['hrPow']
    for i in range(1, duration):
        date = date + timedelta(days=1)
        new_data = read_json(plant_number, date)['hrPow']
        json_data = json_data.append(new_data, ignore_index=True)

    power_data = json_data.interpolate(method='linear')
    return power_data


def get_weather_data(spot_index, date, feature_type, duration=1):
    csv_data = read_csv(spot_index, date, duration=duration)
    weather_data = interpolate_weather(date, csv_data, feature_type)
    weather_data = normalize(weather_data.to_numpy())
    weather_data = np.nan_to_num(weather_data)

    return weather_data


def insert_row(idx, df, df_insert):
    begin = df.iloc[:idx, ]
    end = df.iloc[idx:, ]
    df_insert = pd.DataFrame(list(df_insert.values()), columns=['일시'])
    return pd.concat([begin, df_insert, end])


def interpolate_weather(date, csv_data, feature_type):
    str_date = date.strftime("%Y-%m-%d %H:%M")

    for i in range(23):
        if csv_data.iloc[i:i+1]['일시'].values[0] != str_date:
            row = {'일시': str_date}
            csv_data = insert_row(i, csv_data, row)
        date = date + timedelta(hours=1)
        str_date = date.strftime("%Y-%m-%d %H:%M")

    weather_data = csv_data[feature_type].interpolate(method='linear')
    return weather_data


def normalize(data):
    normalized = (data-min(data))/(max(data)-min(data))
    return normalized


def get_input_data(plant_number, spot_index, starting_date, features, duration):
    input_data = np.empty([len(features) + 1, 24 * duration])

    input_data[0] = get_power_data(plant_number, starting_date, duration)
    for i in range(0, len(features)):
        input_data[i + 1] = get_weather_data(spot_index, starting_date, features[i].value, duration)

    return input_data

### Enum Classes

In [4]:
class FeatureType(enum.Enum):
    TEMPERATURE = "기온(°C)"
    PRECIPITATION = "강수량(mm)"
    WIND_SPEED = "풍속(m/s)"
    WIND_DIRECTION = "풍향(16방위)"
    HUMIDITY = "습도(%)"
    DEW_POINT_TEMPERATURE = "이슬점온도(°C)"
    STEAM_PRESSURE = "현지기압(hPa)"
    SUNSHINE = "일조(hr)"
    VISIBILITY = "시정(10m)"
    GROUND_TEMPERATURE = "지면온도(°C)"
    ATMOSPHERIC_PRESSURE = "증기압(hPa)"
    
class FileType(enum.Enum):
    MODEL = 0
    RESULT = 1

### Manage Dataset

In [5]:
class Dataset:
    def __init__(self, args, features):
        self.plant_number = args.plant
        self.spot_index = args.spot
        self.feature_types = features
        self.start = datetime.strptime(args.start_date, "%Y%m%d")
        self.end = datetime.strptime(args.end_date, "%Y%m%d")
        self.x_frames = args.x_frames
        self.y_frames = args.y_frames
        self.initialize()

    def initialize(self):
        self.duration = (self.end - self.start).days + 1
        self.data = get_input_data(self.plant_number, self.spot_index,
                                   self.start, self.feature_types, self.duration)

    def get_durations(self):
        train_duration = math.floor(self.duration * 0.75)

        val_duration = math.floor(self.duration * 0.125)
        test_duration = math.floor(self.duration * 0.125)
        return train_duration, val_duration, test_duration

    def get_dates(self):
        train_duration, val_duration, test_duration = self.get_durations()
        train_start = self.start
        train_end = train_start + timedelta(days=train_duration - 1)
        val_start = train_end + timedelta(days=1)
        val_end = val_start + timedelta(days=val_duration - 1)
        test_start = val_end + timedelta(days=1)
        test_end = test_start + timedelta(days=test_duration - 1)
        return [train_start, train_end], [val_start, val_end], [test_start, test_end]

    def get_item(self, start):
        index = (start - self.start).days * 24
        X = [self.data[i+1][index:index+(self.x_frames * 24)] for i in range(len(self.feature_types))]
        y = self.data[0][index+(self.x_frames * 24):index+((self.x_frames + self.y_frames) * 24)]
        return np.asarray(X), y

    def get_items(self, start, end):
        len_samples = (end - start).days + 1 - (self.x_frames + self.y_frames) + 1

        X = []
        y = []
        for i in range(len_samples):
            X_item, y_item = self.get_item(start + timedelta(i))
            X.append(X_item)
            y.append(y_item)

        return np.asarray(X), np.asarray(y)

    def get_dataset(self):
        train, val, test = self.get_dates()
        X_train, y_train = self.get_items(train[0], train[1])
        X_val, y_val = self.get_items(val[0], val[1])
        X_test, y_test = self.get_items(test[0], test[1])

        return [X_train, y_train], [X_val, y_val], [X_test, y_test]

In [6]:
def load_dataset(args, features):
    dataset = Dataset(args, features)
    train_set, validation_set, test_set = dataset.get_dataset()
    partition = {'train': train_set, 'val': validation_set, 'test': test_set}
    return partition

### Train, Validate, Test

In [7]:
def generate_model(args):
    with tf.device('/GPU:0'):
        encoder_inputs = Input(shape=(len(args.features), 72))

        encoder = LSTM(args.hid_dim, return_state=True)
        encoder_outputs, state_h, state_c = encoder(encoder_inputs)
        encoder_states = [state_h, state_c]

        repeat_vector = RepeatVector(24)
        repeat_vector_outputs = repeat_vector(encoder_outputs)

        decoder_lstm = LSTM(args.hid_dim, return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_lstm(repeat_vector_outputs, initial_state=encoder_states)

        decoder_dense = Dense(1, activation=args.activation)
        decoder_outputs = decoder_dense(decoder_outputs)

        rmsprop = RMSprop(lr=args.lr)

        model = Model(encoder_inputs, decoder_outputs)
        model.compile(optimizer=rmsprop, loss=tf.keras.losses.MeanSquaredError(), metrics='accuracy')

    return model

In [8]:
def train(partition, args):
    X_train, y_train = partition['train']
    X_val, y_val = partition['val']
    
    batch_size = args.batch_size
    epochs = args.epochs
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=args.early_stop)
    
    model = generate_model(args)
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
              batch_size=batch_size, epochs=epochs, callbacks=[callback])
    model.summary()

    return model, history

In [9]:
def test(partition, model, args):
    X_test, y_test = partition['test']
    
    y_pred = model.predict(X_test)
    y_pred = y_pred.reshape(y_test.shape[0], y_test.shape[1])

    nrmse = compare_nrmse(y_test, y_pred)
    return y_pred, nrmse

In [10]:
def experiment(args, features):
    partition = load_dataset(args, features)
    
    ts = time.time()
    model, history = train(partition, args)
    te = time.time()
    
    print('Took {:2.2f} sec for training the model'.format(te-ts))
    
    y_pred, test_acc = test(partition, model, args)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = history.history['loss']
    result['val_losses'] = history.history['val_loss']
    result['train_accs'] = history.history['accuracy']
    result['val_accs'] = history.history['val_accuracy']
    result['train_acc'] = np.mean(history.history['accuracy'])
    result['val_acc'] = np.mean(history.history['val_accuracy'])
    result['test_nrmse'] = test_acc
    result['y_test'] = np.array_str(partition['test'][1])
    result['y_pred'] = np.array_str(y_pred)
    return vars(args), model, result

### Manage Experiment

In [11]:
def get_filepath(setting, filetype):
    exp_name = setting['exp_name']
    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    
    file_format = ""
    directory = ""
    if filetype == FileType.MODEL:
        file_format = 'h5'
        directory = 'models'
    elif filetype == FileType.RESULT:
        file_format = 'json'
        directory = 'results'
        
    filename = "%s-%s.%s" % (exp_name, hash_key, file_format)
    filepath = os.path.join(get_working_path(), directory, filename)
    return filepath
        
def save_exp_model(setting, model):
    filepath = get_filepath(setting, FileType.MODEL)
    model.save(filepath)

def save_exp_result(setting, result):
    filepath = get_filepath(setting, FileType.RESULT)
    
    result.update(setting)
    with open(filepath, 'w') as f:
        json.dump(result, f)
        
def load_exp_result(exp_name):
    dir_path = os.path.join(get_working_path(), "results")
    filenames = [f for f in listdeir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result)
    return df

In [12]:
# ====== Random Seed Initialization ====== #
seed = 1234
tf.random.set_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "solar power prediction using weather features"

# ====== Data Loading ====== #
args.plant = 126
args.spot = 174
args.start_date = "20190820"
args.end_date = "20191231"
args.batch_size = 64
args.x_frames = 3
args.y_frames = 1

# ====== Model Capacity ===== #
args.hid_dim = 256

# ====== Optimizer & Training ====== #
args.optim = 'RMSprop'
args.activation = 'relu'
args.lss = 'MSE'
args.lr = 0.001
args.epochs = 256
args.early_stop = 30
args.evaluation = 'NRMSE'

# ====== Experiment Variable ====== #
name_var1 = 'features'
list_var1 = [FeatureType.GROUND_TEMPERATURE,
            FeatureType.VISIBILITY,
            FeatureType.WIND_SPEED,
            FeatureType.WIND_DIRECTION,
            FeatureType.TEMPERATURE,
            FeatureType.STEAM_PRESSURE,
            FeatureType.ATMOSPHERIC_PRESSURE,
            FeatureType.HUMIDITY,
            FeatureType.DEW_POINT_TEMPERATURE]

for i in range(len(list_var1)):
    sub_list = list_var1[:i+1]
    setattr(args, name_var1, str(sub_list))
    print(args)
    
    setting, model, result = experiment(deepcopy(args), sub_list)
    save_exp_model(setting, model)
    save_exp_result(setting, result)

Namespace(activation='relu', batch_size=64, early_stop=30, end_date='20191231', epochs=256, evaluation='NRMSE', exp_name='solar power prediction using weather features', features="[<FeatureType.GROUND_TEMPERATURE: '지면온도(°C)'>]", hid_dim=256, lr=0.001, lss='MSE', optim='RMSprop', plant=126, spot=174, start_date='20190820', x_frames=3, y_frames=1)
Epoch 1/256
2/2 [==============================] - 1s 290ms/step - loss: 567859584.0000 - accuracy: 0.3827 - val_loss: 674779904.0000 - val_accuracy: 0.0833
Epoch 2/256
2/2 [==============================] - 0s 14ms/step - loss: 567639104.0000 - accuracy: 0.0077 - val_loss: 674443264.0000 - val_accuracy: 0.0192
Epoch 3/256
2/2 [==============================] - 0s 14ms/step - loss: 567449792.0000 - accuracy: 0.0000e+00 - val_loss: 674349440.0000 - val_accuracy: 0.0128
Epoch 4/256
2/2 [==============================] - 0s 13ms/step - loss: 567386560.0000 - accuracy: 0.0000e+00 - val_loss: 674300928.0000 - val_accuracy: 0.0128
Epoch 5/256
2/2 [==

2/2 [==============================] - 0s 13ms/step - loss: 566827840.0000 - accuracy: 0.0000e+00 - val_loss: 673691200.0000 - val_accuracy: 0.0000e+00
Epoch 45/256
2/2 [==============================] - 0s 13ms/step - loss: 566815616.0000 - accuracy: 0.0000e+00 - val_loss: 673677440.0000 - val_accuracy: 0.0000e+00
Epoch 46/256
2/2 [==============================] - 0s 16ms/step - loss: 566803200.0000 - accuracy: 0.0000e+00 - val_loss: 673664320.0000 - val_accuracy: 0.0032
Epoch 47/256
2/2 [==============================] - 0s 13ms/step - loss: 566791168.0000 - accuracy: 0.0000e+00 - val_loss: 673651136.0000 - val_accuracy: 0.0128
Epoch 48/256
2/2 [==============================] - 0s 13ms/step - loss: 566778944.0000 - accuracy: 8.5911e-04 - val_loss: 673637312.0000 - val_accuracy: 0.0064
Epoch 49/256
2/2 [==============================] - 0s 14ms/step - loss: 566766592.0000 - accuracy: 0.0198 - val_loss: 673623616.0000 - val_accuracy: 0.0000e+00
Epoch 50/256
2/2 [=====================

Epoch 96/256
2/2 [==============================] - 0s 14ms/step - loss: 566193344.0000 - accuracy: 0.1095 - val_loss: 672994240.0000 - val_accuracy: 0.1859
Epoch 97/256
2/2 [==============================] - 0s 13ms/step - loss: 566181056.0000 - accuracy: 0.1327 - val_loss: 672980928.0000 - val_accuracy: 0.1891
Epoch 98/256
2/2 [==============================] - 0s 13ms/step - loss: 566168896.0000 - accuracy: 0.1276 - val_loss: 672964736.0000 - val_accuracy: 0.1474
Epoch 99/256
2/2 [==============================] - 0s 13ms/step - loss: 566156992.0000 - accuracy: 0.1280 - val_loss: 672951936.0000 - val_accuracy: 0.1667
Epoch 100/256
2/2 [==============================] - 0s 13ms/step - loss: 566144832.0000 - accuracy: 0.1302 - val_loss: 672938112.0000 - val_accuracy: 0.1667
Epoch 101/256
2/2 [==============================] - 0s 13ms/step - loss: 566132416.0000 - accuracy: 0.1405 - val_loss: 672924032.0000 - val_accuracy: 0.1250
Epoch 102/256
2/2 [==============================] - 0s 

Epoch 148/256
2/2 [==============================] - 0s 14ms/step - loss: 565558272.0000 - accuracy: 0.1581 - val_loss: 672291840.0000 - val_accuracy: 0.2083
Epoch 149/256
2/2 [==============================] - 0s 14ms/step - loss: 565545984.0000 - accuracy: 0.1559 - val_loss: 672278144.0000 - val_accuracy: 0.2019
Epoch 150/256
2/2 [==============================] - 0s 13ms/step - loss: 565533760.0000 - accuracy: 0.1551 - val_loss: 672264896.0000 - val_accuracy: 0.2083
Epoch 151/256
2/2 [==============================] - 0s 16ms/step - loss: 565521728.0000 - accuracy: 0.1559 - val_loss: 672251328.0000 - val_accuracy: 0.1955
Epoch 152/256
2/2 [==============================] - 0s 13ms/step - loss: 565509696.0000 - accuracy: 0.1508 - val_loss: 672237888.0000 - val_accuracy: 0.1667
Epoch 153/256
2/2 [==============================] - 0s 13ms/step - loss: 565497536.0000 - accuracy: 0.1482 - val_loss: 672224192.0000 - val_accuracy: 0.1667
Epoch 154/256
2/2 [==============================] -

Epoch 200/256
2/2 [==============================] - 0s 15ms/step - loss: 564923968.0000 - accuracy: 0.1392 - val_loss: 671592128.0000 - val_accuracy: 0.0929
Epoch 201/256
2/2 [==============================] - 0s 14ms/step - loss: 564912064.0000 - accuracy: 0.1387 - val_loss: 671578240.0000 - val_accuracy: 0.1410
Epoch 202/256
2/2 [==============================] - 0s 14ms/step - loss: 564899456.0000 - accuracy: 0.1353 - val_loss: 671564224.0000 - val_accuracy: 0.0962
Epoch 203/256
2/2 [==============================] - 0s 14ms/step - loss: 564886848.0000 - accuracy: 0.1387 - val_loss: 671550784.0000 - val_accuracy: 0.0865
Epoch 204/256
2/2 [==============================] - 0s 14ms/step - loss: 564874560.0000 - accuracy: 0.1409 - val_loss: 671537152.0000 - val_accuracy: 0.1346
Epoch 205/256
2/2 [==============================] - 0s 13ms/step - loss: 564862272.0000 - accuracy: 0.1392 - val_loss: 671523904.0000 - val_accuracy: 0.0897
Epoch 206/256
2/2 [==============================] -

...

Epoch 10/256
2/2 [==============================] - 0s 13ms/step - loss: 567246976.0000 - accuracy: 0.0000e+00 - val_loss: 674151168.0000 - val_accuracy: 0.0000e+00
Epoch 11/256
2/2 [==============================] - 0s 14ms/step - loss: 567233024.0000 - accuracy: 0.0000e+00 - val_loss: 674135744.0000 - val_accuracy: 0.0000e+00
Epoch 12/256
2/2 [==============================] - 0s 14ms/step - loss: 567219136.0000 - accuracy: 0.0000e+00 - val_loss: 674121024.0000 - val_accuracy: 0.0000e+00
Epoch 13/256
2/2 [==============================] - 0s 13ms/step - loss: 567205632.0000 - accuracy: 0.0000e+00 - val_loss: 674105728.0000 - val_accuracy: 0.0000e+00
Epoch 14/256
2/2 [==============================] - 0s 14ms/step - loss: 567192000.0000 - accuracy: 0.0000e+00 - val_loss: 674091456.0000 - val_accuracy: 0.0000e+00
Epoch 15/256
2/2 [==============================] - 0s 16ms/step - loss: 567179200.0000 - accuracy: 0.0000e+00 - val_loss: 674077760.0000 - val_accuracy: 0.0000e+00
Epoch 16/2

Epoch 60/256
2/2 [==============================] - 0s 15ms/step - loss: 566628288.0000 - accuracy: 0.0000e+00 - val_loss: 673471232.0000 - val_accuracy: 0.0000e+00
Epoch 61/256
2/2 [==============================] - 0s 13ms/step - loss: 566616384.0000 - accuracy: 0.0000e+00 - val_loss: 673457728.0000 - val_accuracy: 0.0000e+00
Epoch 62/256
2/2 [==============================] - 0s 16ms/step - loss: 566604352.0000 - accuracy: 0.0000e+00 - val_loss: 673444800.0000 - val_accuracy: 0.0000e+00
Epoch 63/256
2/2 [==============================] - 0s 14ms/step - loss: 566592512.0000 - accuracy: 0.0000e+00 - val_loss: 673431488.0000 - val_accuracy: 0.0000e+00
Epoch 64/256
2/2 [==============================] - 0s 13ms/step - loss: 566580352.0000 - accuracy: 0.0000e+00 - val_loss: 673417984.0000 - val_accuracy: 0.0000e+00
Epoch 65/256
2/2 [==============================] - 0s 14ms/step - loss: 566568256.0000 - accuracy: 0.0000e+00 - val_loss: 673404800.0000 - val_accuracy: 0.0000e+00
Epoch 66/2

2/2 [==============================] - 0s 13ms/step - loss: 566035968.0000 - accuracy: 0.0000e+00 - val_loss: 672817600.0000 - val_accuracy: 0.0000e+00
Epoch 110/256
2/2 [==============================] - 0s 17ms/step - loss: 566023680.0000 - accuracy: 0.0000e+00 - val_loss: 672804480.0000 - val_accuracy: 0.0000e+00
Epoch 111/256
2/2 [==============================] - 0s 14ms/step - loss: 566011584.0000 - accuracy: 0.0000e+00 - val_loss: 672790656.0000 - val_accuracy: 0.0064
Epoch 112/256
2/2 [==============================] - 0s 14ms/step - loss: 565999232.0000 - accuracy: 0.0000e+00 - val_loss: 672777472.0000 - val_accuracy: 0.0096
Epoch 113/256
2/2 [==============================] - 0s 15ms/step - loss: 565987200.0000 - accuracy: 0.0000e+00 - val_loss: 672763968.0000 - val_accuracy: 0.0160
Epoch 114/256
2/2 [==============================] - 0s 14ms/step - loss: 565974912.0000 - accuracy: 0.0017 - val_loss: 672750528.0000 - val_accuracy: 0.0288
Epoch 115/256
2/2 [===================

Epoch 161/256
2/2 [==============================] - 0s 14ms/step - loss: 565404800.0000 - accuracy: 0.1259 - val_loss: 672122304.0000 - val_accuracy: 0.1282
Epoch 162/256
2/2 [==============================] - 0s 13ms/step - loss: 565392832.0000 - accuracy: 0.1383 - val_loss: 672108800.0000 - val_accuracy: 0.1250
Epoch 163/256
2/2 [==============================] - 0s 14ms/step - loss: 565380736.0000 - accuracy: 0.1319 - val_loss: 672095936.0000 - val_accuracy: 0.1250
Epoch 164/256
2/2 [==============================] - 0s 13ms/step - loss: 565368832.0000 - accuracy: 0.1362 - val_loss: 672082304.0000 - val_accuracy: 0.1250
Epoch 165/256
2/2 [==============================] - 0s 14ms/step - loss: 565356544.0000 - accuracy: 0.1250 - val_loss: 672068992.0000 - val_accuracy: 0.1250
Epoch 166/256
2/2 [==============================] - 0s 14ms/step - loss: 565344640.0000 - accuracy: 0.1267 - val_loss: 672055936.0000 - val_accuracy: 0.1250
Epoch 167/256
2/2 [==============================] -

Epoch 213/256
2/2 [==============================] - 0s 13ms/step - loss: 564776256.0000 - accuracy: 0.1735 - val_loss: 671428672.0000 - val_accuracy: 0.1667
Epoch 214/256
2/2 [==============================] - 0s 13ms/step - loss: 564763712.0000 - accuracy: 0.1757 - val_loss: 671414976.0000 - val_accuracy: 0.1667
Epoch 215/256
2/2 [==============================] - 0s 17ms/step - loss: 564751424.0000 - accuracy: 0.1757 - val_loss: 671401664.0000 - val_accuracy: 0.1667
Epoch 216/256
2/2 [==============================] - 0s 14ms/step - loss: 564739392.0000 - accuracy: 0.1735 - val_loss: 671388544.0000 - val_accuracy: 0.1667
Epoch 217/256
2/2 [==============================] - 0s 14ms/step - loss: 564727424.0000 - accuracy: 0.1671 - val_loss: 671375360.0000 - val_accuracy: 0.1667
Epoch 218/256
2/2 [==============================] - 0s 14ms/step - loss: 564715392.0000 - accuracy: 0.1692 - val_loss: 671361792.0000 - val_accuracy: 0.1667
Epoch 219/256
2/2 [==============================] -